This is Assignment 1 of the Udacity course on Deep Learning

Preprocess notMNIST data and train a simple logistic regression model on it

Data are read and pickled using code available here:

For details see: 

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/1_notmnist.ipynb

I add this notebook for completeness to my repo too

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import time
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from six.moves import cPickle as pickle

%matplotlib inline

In [ ]:
# Display random image from chosen dir

choose_dir = 'notMNIST_small/A'
path_to_dir = os.path.join(data_root, choose_dir)
images = os.listdir(path_to_dir)
display(Image(filename=os.path.join(path_to_dir, random.choice(images))))

In [ ]:
# Read pickled dataset

data_root = 'notMNIST/'
with open(os.path.join(data_root, 'notMNIST.pickle'), 'rb') as f:
    data = pickle.load(f)
    
train_dataset = data['train_dataset']
train_labels = data['train_labels']
valid_dataset = data['valid_dataset']
valid_labels= data['valid_labels']
test_dataset = data['test_dataset']
test_labels = data['test_labels']

In [ ]:
# Check if it still looks good after pickling using matplotlib

sample_image = random.choice(train_dataset)
plt.imshow(sample_image)

In [ ]:
# Check for overlapping samples

def count_overlap(dataset_1, dataset_2):
    start = time.clock()
    hashed_1 = {hash(tuple(image.ravel())) for image in dataset_1}
    hashed_2 = {hash(tuple(image.ravel())) for image in dataset_2}
    n_overlaps = len(set(hashed_1).intersection(hashed_2))
    return n_overlaps, time.clock()-start

c1 = count_overlap(train_dataset, valid_dataset)
c2 = count_overlap(valid_dataset, test_dataset)
c3 = count_overlap(train_dataset, test_dataset)

print('overlap between train and valid {0:06d} evaluated in {0:.3f}'.format(c1[0], c1[1]))
print('overlap between valid and test {0:06d} evaluated in {0:.3f}'.format(c2[0], c2[1]))
print('overlap between train and test {0:06d} evaluated in {0:.3f}'.format(c3[0], c3[1]))

In [ ]:
def there(img):
    return tuple(img.ravel())

def back(tup, size_x=28, size_y=28):
    return np.array(tup).reshape((size_x, size_y))

# Clear from overlapping samples
def clear_overlap(dataset, dataset_to_clear):
    start = time.clock()
    set_1 = {there(img) for img in dataset}
    set_to_clear = {there(img) for img in dataset_to_clear}
    cleared_aux = set_to_clear.difference(set_1)
    print('took {}'.format(time.clock()-start))
    return np.array([back(elem) for elem in cleared_aux])

valid_dataset_c = clear_overlap(train_dataset, valid_dataset)

test_dataset_c = clear_overlap(valid_dataset, test_dataset)
test_dataset_c = clear_overlap(train_dataset, test_dataset_c)

print(valid_dataset.shape, valid_dataset_c.shape)
print(test_dataset.shape, test_dataset_c.shape)

In [ ]:
# Train Logistic Regression Classifier on the dataset

# Prepare data
def prepare(dataset):
    n, w, h = dataset.shape
    return np.reshape(dataset,(n, w*h))

X_train = prepare(train_dataset)
y_train = train_labels

X_test = prepare(test_dataset_c)
y_test = test_labels

model = LogisticRegression(
    multi_class='multinomial', 
    solver='lbfgs', 
    random_state=42, 
    verbose=1, 
    max_iter=1000, 
    n_jobs=-1,
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_test, y_pred)